# Embedtree2
Notebook to compile the so far research of nvgtt.

In [44]:
import requests
import nltk
from bs4 import BeautifulSoup
import re
import urllib
import networkx as nx
import sys

from ThreadPool import ThreadPool

from wikipydia import dict_storage, wikidb
DictStorage = dict_storage.DictStorage

#from graph_features import *

In [45]:
wiki_db = wikidb.WikiDb()

In [46]:
last_percent_reported = None
def download_progress_hook(count, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent

In [47]:
wikisyn = DictStorage("wikisyn") #Storage for link synoms

In [48]:
def get_wiki_article_by_href(href):
    wikiart, downloaded = wiki_db.get_article_by_href(href)
    #Populate wikisyn
    for link_href, link_text in wikiart.links():
        if not link_href in wikisyn:
            wikisyn[link_href] = set()
        wikisyn[link_href].add(link_text)
        
    return wikiart

In [49]:
def save_everything():
    wiki_db.save()
    wikisyn.save()

In [50]:
def test_get_wiki_article_by_href(href):
    wikiart = get_wiki_article_by_href(href)
    save_everything()
    return wikiart

#print(test_get_wiki_article_by_href("c%2b%2b").title)


In [51]:
#Function to download bunch of wikipedia pages at once if they are not present
check_and_download__done = 0
def check_and_download(pages):
    global check_and_download__done
    check_and_download__done = 0
    
    n_tasks = len(pages)
    
    print("Tasks to go: " + str(n_tasks))
    
    # Function to be executed in a thread
    def download_stuff(page):
        global check_and_download__done
        try:
            get_wiki_article_by_href(page)
        except requests.exceptions.Timeout:
            print("Failed to get page " + page + ". Timed out.")
        finally:
            check_and_download__done += 1
            download_progress_hook(check_and_download__done, n_tasks)
            #percentage = round(check_and_download__done * 100 / n_tasks)
            #if percentage % 5 == 0:
                #print("{0}%.....".format(percentage), end="")
            #print("Done " + str(check_and_download__done) + "/" + str(n_tasks))

    # Instantiate a thread pool with 5 worker threads
    pool = ThreadPool(50)

    pool.map(download_stuff, pages)
    pool.wait_completion()
    
    print("\nFinishing downloading. Done tasks: " + str(check_and_download__done) + "/" + str(n_tasks))

In [52]:
#check_and_download(["Node.js"])

In [53]:
#target_page = urllib.quote("JavaScript")
#target_id = get_pageid(target_page)
#target_links = pageid_to_page_links[target_id]
#target_links_ids = set()

#for i, link in enumerate(target_links):
    #print("Working on " + link + ". " + str((i+1)) + "/" + str(len(target_links)))
    #target_links_ids.add(get_pageid(link))


In [54]:
def print_sorted_list(data, key, reverse=False):
    for k, v in sorted(data, key=key, reverse=reverse):
        print(k,v)  

In [55]:
def get_links_score(page):
    """Function to cross a list of links with a text, setting scores."""
    
    wikiart = get_wiki_article_by_href(page)
    
    pageid = wikiart.page_id
    page_links = [link_href for link_href, link_text in wikiart.links()]
    page_text = wikiart.text()
    
    links_score = dict()
    
    norm_fact = 0
    
    #Ensure all page links are present
    #check_and_download(page_links)
    
    for link_href in page_links:
        links_score[link_href] = 0
        for l_text in wikisyn[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            links_score[link_href] += len(matches) 
            norm_fact += len(matches)
            
    norm_links_score = dict(map(lambda a: [a[0], float(a[1])/norm_fact], links_score.items()))
            
    return norm_links_score

#v_sum = 0
#links_score = get_links_score("MQTT")
#for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    #print(k,wikisyn[k],v)
    #v_sum += v
#print v_sum

In [56]:
#print_sorted_list(get_links_score("MQTT").items(), lambda a: a[1], True)

In [57]:
def get_node_edges_scores(page_href, top=-1):
    """Function to get node edges to be placed in the graph. """
    
    edges = dict()
    
    wikiart = get_wiki_article_by_href(page_href)
    
    #Get main page data
    page_id = wikiart.page_id
    page_title = wikiart.title
    page_links = get_links_score(page_href).items()
    
    if top > -1:
        sorted_page_links = sorted(page_links, key=lambda a: a[1], reverse=True)
        page_links = sorted_page_links[:top]
        #print(page_links)
    
    for i, (link_href, score) in enumerate(page_links):
        
        #print("Working on link {0} {1}/{2}".format(link_href, i+1, len(page_links)))
        
        link_art = get_wiki_article_by_href(link_href)
        
        link_id = link_art.page_id
        link_title = link_art.title
        
        #If there is already a title already place, sum the scores
        if (page_title, link_title) in edges:
            edges[(page_title, link_title)] += score
        else:
            edges[(page_title, link_title)] = score        
        
    return edges
    

#edges_scores = get_node_edges_scores("TensorFlow")
#print_sorted_list(edges_scores.items(), lambda a:a[1], True)

#save_everything()

#CREATE METHOD TO CREATE GRAPH BASED ON DEEPNESS --DONE
#MAYBE PLACE STOP CONDITION TO NOT DOWNLOAD EVERY LINK --DONE
#CHECK WHETHER WIKISYN IS REALLY GOOD BECAUSE OF ERRORS. MAYBE KEEP TRACK HOW MANY TIMES EACH WORD APPEARS

In [73]:
def get_graph_edges(seed_href, deepness, top=-1):
    if deepness > 3:
        raise Exception("Not allowed more than 2 of deepness")
    
    edges = list()
    
    seed_title = get_wiki_article_by_href(seed_href).title
    
    done_titles = list()
    todo_queue = list()
    
    todo_queue.append(seed_title)
    
    for i in range(deepness):
        print("Working on batch {0} of {1}".format(i+1, deepness))
        
        current_todo_queue = todo_queue
        todo_queue = list()
        
        print("queue size: {0}".format(len(current_todo_queue)))
        
        while len(current_todo_queue) > 0:
            
            next_title = current_todo_queue.pop() 
        
            if next_title in done_titles:
                continue
        
            edges_scores = get_node_edges_scores(urllib.parse.quote(next_title), top)
        
            done_titles.append(next_title)
            
            #Sort edges scores and take the top 5
            #sorted_edges = sorted(edges_scores.items(), key=lambda a: a[1], reverse=True)[:5]
            
            #Save new edges and next to do titles
            for edge, score in edges_scores.items():
                edges.append((edge, score))
                
                if not edge[1] in done_titles:
                    todo_queue.append(edge[1])
    
    save_everything() 
    return edges           

    
#edges = get_graph_edges("Deep learning", 2, 5)
#save_everything()

#for e in edges:
    #print(e)


In [74]:
def get_graph(edges):
    #Create a directed graph
    graph = nx.DiGraph()
    for edge in edges:
        graph.add_edge(edge[0], edge[1])
    return graph

In [75]:
def extract_path_tuples(path):
    """Function that generates all path tuples from a list path."""
    path_tuples = []
    for i, _ in enumerate(path):
        if i == 0:
            continue
        path_tuples.append((path[i-1], path[i]))

    return path_tuples

def get_features(graph, seed_node, cutoff, deep_rank_scores=None):
    """
    Function to compute the prereq probabilities for every node.
    Compute deeprank and bidirection rank. TO BE EXPLAINED

    Returns: bidir_probs, deeprank_probs, n_paths, min_depths, max_depths

    """

    #Compute number of edges per node
    nodes_edges = dict()
    for n1, n2 in graph.edges():
        #init nodes dict if not initiated
        if not n1 in nodes_edges:
            nodes_edges[n1] = 0
        nodes_edges[n1] += 1

    #Compute initial edges probabilities
    #For bidirection rank we set 1 in case the edge is unidirectional and 0.5 in case bidirectional.
    #For deeprank we compute the fraction of the edge over all the edges in the same node.

    ### Later we need to create other methods to compute proper distribution for the cases above. ### 

    bidir_edges_values = dict()
    deeprank_edges_values = dict()

    #edges_values = dict()
    for n1, n2 in graph.edges():
        if deep_rank_scores == None:
            deeprank_edges_values[(n1, n2)] = 1.0 / nodes_edges[n1]
        
        if graph.has_edge(n2, n1):
            bidir_edges_values[(n1, n2)] = 0.5
        else:
            bidir_edges_values[(n1, n2)] = 1

    if deep_rank_scores != None:
        deeprank_edges_values = dict(deep_rank_scores)

    #Now compute all the paths to the target seed_node and sequence probabilities to each path
    bidir_probs = dict() #Probabilities of reach seed_node from each node based on bidir values
    deeprank_probs = dict() #Probabilities of reach seed_node from each node based on deeprank values
    min_depths = dict() #Each node min depth
    max_depths = dict() #Each node max depth
    ns_paths = dict() #Each node number of paths

    # create dicts for every feature extracted
    # try get insights from kmeans
    # try to find something to deploy FAST (the energy applied must be low!)

    n_nodes = nx.number_of_nodes(graph)

    #Iterate thru all the graph nodes
    for i, node in enumerate(graph.nodes()):

        print("Working on node {0}/{1}".format(i+1,n_nodes))

        #Init min max depth
        min_depth = cutoff + 2
        max_depth = 0

        #Skip seed_node since we do not want verify paths to itself
        if node == seed_node:
            continue

        n_paths = 0
        total_bidir_prob = 0
        total_deeprank_prob = 0

        for path in nx.all_simple_paths(graph, source=seed_node, target=node, cutoff=cutoff):

            n_paths += 1
            partial_bidir_prob = 1.0
            partial_deeprank_prob = 1.0

            #Computes min-max depth
            max_depth = max(max_depth, len(path))
            min_depth = min(min_depth, len(path))

            #Iterate the path tuples
            for i, edge_tuple in enumerate(extract_path_tuples(path)):
                partial_bidir_prob *= bidir_edges_values[edge_tuple]
                partial_deeprank_prob *= deeprank_edges_values[edge_tuple]

            total_bidir_prob += partial_bidir_prob
            total_deeprank_prob += partial_deeprank_prob

        #After processing all node paths, save the final values    
        bidir_probs[node] = total_bidir_prob / n_paths
        deeprank_probs[node] = total_deeprank_prob
        min_depths[node] = min_depth
        max_depths[node] = max_depth
        ns_paths[node] = n_paths 

    return bidir_probs, deeprank_probs, ns_paths, min_depths, max_depths

In [76]:
target_article = "AngularJS"

scored_edges = get_graph_edges(target_article, 3, 5)
edges = [edge for edge,score in scored_edges]

print(scored_edges)
print(edges)
graph = get_graph(edges)

bidir_probs, deeprank_probs, ns_paths, min_depths, max_depths = get_features(graph, target_article, 3, scored_edges)

print("\n")
for title, dr in sorted(deeprank_probs.items(), key=lambda a: a[1], reverse=True):
    print(title, dr, bidir_probs[title], ns_paths[title], min_depths[title], max_depths[title])
    
MUST PLAY WITH PARAMETERS AND IMPLEMENT PARALEL DOWNLOAD ON GET GRAPH EDGES

Working on batch 1 of 3
queue size: 1
Working on batch 2 of 3
queue size: 5
Working on batch 3 of 3
queue size: 22
[(('AngularJS', 'Scope (computer science)'), 0.08514492753623189), (('AngularJS', 'Angular (application platform)'), 0.07971014492753623), (('AngularJS', 'Angular'), 0.06702898550724638), (('AngularJS', 'Variable (computer science)'), 0.025362318840579712), (('AngularJS', 'JSON'), 0.025362318840579712), (('JSON', 'JavaScript'), 0.05332167832167832), (('JSON', 'String (computer science)'), 0.038461538461538464), (('JSON', 'Web browser'), 0.03496503496503497), (('JSON', 'Programming language'), 0.02972027972027972), (('JSON', 'File format'), 0.027097902097902096), (('Variable (computer science)', 'Programming language'), 0.06622516556291391), (('Variable (computer science)', 'Computer programming'), 0.05392620624408704), (('Variable (computer science)', 'Scope (computer science)'), 0.0510879848628193), (('Variable (computer science)', 'Value (computer science)'), 0.040681173

In [238]:
#print(page_data['full'].get_text())